In [ ]:
import paths
import yaml
import os
import copy
import numpy as np
import numpy.random as npr
import scipy.optimize as spo
import scipy.linalg as spl
from matplotlib import pyplot as plt, collections as mc, patches as mpatches, cm, ticker
from sdfs.geom_mrst import GeomMRST
from sdfs.bc_mrst import BCMRST
from sdfs.darcy import DarcyExp
from sdfs.tpfa import TPFA
from sdfs.dasa import DASAExpLM, DASAExpLMWithFlux
from time import perf_counter
import ckli.mapest as mapest
import ckli.ckliest_l2reg as ckliest
import h5py
import GPy

In [ ]:
plt.rc('text', usetex=True)
plt.rc('image', cmap='plasma')

def plot_patch(patches, values, fig, ax, points, title, cmin=None, cmax=None, cb=False):
    p = mc.PatchCollection(patches, cmap=cm.jet)
    p.set_array(values)
    p.set_clim([cmin, cmax])
    ax.add_collection(p)
    if points is not None:
        ax.plot(*points, 'ko', markersize=0.5)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.autoscale(tight=True)
    #ax.set_title(title)
    if cb:
        fig.colorbar(p, ax=ax)
    return p

In [ ]:
# Parameters
seed = 0
num_trials = 1
res_fac = 1
resolution = '1x'
resolution_fine = '16x'
NYobs = 100
NYlearn = NYobs
NYrefobs = 50
NYxi = 1000
Nuxi = 1000
Nens = 5000
beta_ckli = 1e1
Ygamma_ckli = 1e-4
ugamma_ckli = 1e-4
gamma_map = 1e-6
std_dev_ref = 1.0
cor_len_ref = 0.1
Neumann_sd = 0
lsq_method = 'trf'
data_path = '../data/'
results_path = '../results/'
figures_path = '../figures/'
geom_filename = data_path + f'geom/geom_{resolution}.mat'
geom_fine_filename = data_path + f'geom/geom_{resolution_fine}.mat'
bc_filename = data_path + f'bc/bc_{resolution}.mat'
conduct_filename = data_path + f'RF2/conduct_log_RF2_{NYrefobs}_{resolution}.mat'
well_cells_filename = data_path + f'well_cells/well_cells_{resolution}.mat'
yobs_filename = data_path + f'yobs/yobs_{NYobs}_{resolution}.npy'
yobs_fine_filename = data_path + f'yobs/yobs_{NYobs}_{resolution_fine}.npy'
ref = f"Yref=RF2_{NYrefobs}_{resolution}"

In [ ]:
Yfac = 7.0 # Rescaling factor for log-conductivity. Must be applied to Yref and the BCs

geom = GeomMRST(geom_filename)
bc = BCMRST(geom, bc_filename)
bc.rescale('N', Yfac)
prob = DarcyExp(TPFA(geom, bc), None)

Nc = geom.cells.num
Ninf = geom.faces.num_interior
print(f'Ninf = {Ninf}, Nc = {Nc}')

In [ ]:
patches = [mpatches.Polygon(v, closed=True) for v in geom.nodes.coords.T[geom.cells.nodes.T, :]]

In [ ]:
# Observations
rs = npr.RandomState(seed)

# Read stochastic model from GPML output
with h5py.File(conduct_filename, 'r') as f:
    Yref = f.get('mu')[:].ravel() - Yfac
    xrefYobs = f.get('xYobs')[:]

uref = prob.randomize_bc('N', Neumann_sd).solve(Yref)

# u observations
with h5py.File(well_cells_filename, 'r') as f:
    iuobs = f.get('well_cells')[:].ravel() - 1
uobs = uref[iuobs]
Nuobs = iuobs.size

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
p = plot_patch(patches, Yref + Yfac, fig, ax, xrefYobs, 'Yref', 0, 12)
cbar = fig.colorbar(p, ax=ax)
cbar.ax.tick_params(labelsize='30')
cbar.locator = ticker.MaxNLocator(nbins=7)
cbar.update_ticks()
fig.tight_layout()
fig.savefig(figures_path + f'ref/Yref_{ref}.pdf', dpi=300)

In [ ]:
rl2e = lambda yest, yref : spl.norm(yest - yref, 2) / spl.norm(yref, 2)
infe = lambda yest, yref : spl.norm(yest - yref, np.inf)

In [ ]:
if os.path.exists(yobs_filename):
    print(f"iYobs set read from file {yobs_filename}")
    iYobs = np.load(yobs_filename)
elif os.path.exists(yobs_fine_filename):
    print(f"iYobs set read from file {yobs_fine_filename} and randomly selected nearby cell")
    iYobs_fine = np.load(yobs_fine_filename)
    geom_fine = GeomMRST(geom_fine_filename)
    iYobs = np.array([geom.anyCellsWithin(geom_fine.nodes.coords.T[geom_fine.cells.nodes.T[iYobs_fine[t]]]) for t in range(num_trials)])
    np.save(yobs_filename, iYobs)
else:
    print(f"iYobs set randomly generated and saved to {yobs_filename}")
    iYobs = np.array([np.sort(rs.choice(Nc, NYobs, replace=False)) for _ in range(num_trials)])
    np.save(yobs_filename, iYobs)
print(f"{iYobs.shape=}")
print(iYobs)

In [ ]:
exp = f'NY={NYobs}_Nu={iuobs.size}_{NYlearn=}_{Nuxi=}_{NYxi=}_beta={beta_ckli}_gamma={ugamma_ckli}_Neumann_sd={Neumann_sd}_{lsq_method=}_h1reg_{ref}'
print(exp)

In [ ]:
timings = np.zeros((num_trials, 6))
nfevs = np.zeros((num_trials, 3), dtype=int)
rel_errors = np.zeros((num_trials, 4))
abs_errors = np.zeros((num_trials, 4))

Yobs = np.zeros((num_trials, NYobs))
Ypred = np.zeros((num_trials, Nc))
CYpred = np.zeros((num_trials, Nc, Nc))
umean = np.zeros((num_trials, Nc))
Cu = np.zeros((num_trials, Nc, Nc))
upred = np.zeros((num_trials, Nc))
Cupred = np.zeros((num_trials, Nc, Nc))

PsiY = np.zeros((num_trials, Nc, NYxi))
LambdaY = np.zeros((num_trials, NYxi))
Psiu = np.zeros((num_trials, Nc, Nuxi))
Lambdau = np.zeros((num_trials, Nuxi))

Yxi = np.zeros((num_trials, NYxi))
uxi = np.zeros((num_trials, Nuxi))
Yest = np.zeros((num_trials, Nc))
uest = np.zeros((num_trials, Nc))
Yest_MAPH1 = np.zeros((num_trials, Nc))
if Neumann_sd != 0:
    Nq = np.count_nonzero(bc.kind == 'N')
    q_MAPH1 = np.zeros((num_trials, Nq))

In [ ]:
for t in range(num_trials):
    Yobs[t] = Yref[iYobs[t]]

    ts = perf_counter()
    klearn = GPy.kern.sde_Matern52(input_dim=2, variance=std_dev_ref**2, lengthscale=cor_len_ref)
    mYlearn = GPy.models.GPRegression(geom.cells.centroids[:, iYobs[t]].T, Yobs[t, :,None], klearn, noise_var=np.sqrt(np.finfo(float).eps))
    mYlearn.optimize(messages=True, ipython_notebook=False)
    print(f"{klearn.lengthscale.values[0]=}")
    print(f"{np.sqrt(klearn.variance.values[0])=}")

    mYref = GPy.models.GPRegression(geom.cells.centroids[:, iYobs[t]].T, Yobs[t, :, None], mYlearn.kern, noise_var=np.sqrt(np.finfo(float).eps))
    Ypred[t], CYpred[t] = (lambda x, y : (x.ravel(), y))(*mYref.predict_noiseless(geom.cells.centroids.T, full_cov=True))
    timings[t, 0] = perf_counter() - ts

print(f"GPR: {timings[:, 0]} s")

In [ ]:
for t in range(num_trials):
    # Compute GP model for u
    ts = perf_counter()
    umean[t], Cu[t] = ckliest.smc_gp(Ypred[t], CYpred[t], Nens, copy.deepcopy(prob), rs, randomize_bc=True, randomize_scale=Neumann_sd)
    upred[t], Cupred[t] = ckliest.gpr(umean[t], Cu[t], uobs, iuobs)
    timings[t, 1] = perf_counter() - ts

print(f"Monte Carlo: {timings[:, 1]} s")

In [ ]:
# PICKLE models
Ym = Ypred
CYm = CYpred
um = umean #or change to upred
Cum = Cu #or change to Cupred

rel_errors[:, 0] = np.array([rl2e(Ym[t], Yref) for t in range(num_trials)])
abs_errors[:, 0] = np.array([infe(Ym[t], Yref) for t in range(num_trials)])

print(f"GPR\tRelative error: {rel_errors[:, 0]}")
print(f"GPR\tInfinity error: {abs_errors[:, 0]}")

In [ ]:
for t in range(num_trials):
    ts = perf_counter()
    PsiY[t], LambdaY[t] = ckliest.KL_via_eigh(CYm[t], NYxi)
    Psiu[t], Lambdau[t] = ckliest.KL_via_eigh(Cum[t], Nuxi)
    timings[t, 2] = perf_counter() - ts

print(f"eigendecomposition: {timings[:, 2]} s")

In [ ]:
# PICKLE estimate
ssv = None if Neumann_sd == 0 else np.delete(np.arange(Nc), np.unique(geom.cells.to_hf[2*geom.faces.num_interior:][bc.kind == 'N']))

for t in range(num_trials):
    res = ckliest.LeastSqRes(NYxi, Ym[t], PsiY[t], Nuxi, um[t], Psiu[t], prob, ugamma_ckli, Ygamma_ckli, res_fac, iuobs, uobs, iYobs[t], Yobs[t], beta_ckli, ssv=ssv)
    x0 = np.zeros(Nuxi + NYxi)
        
    ts = perf_counter()
    sol = spo.least_squares(res.val, x0, jac=res.jac, method=lsq_method, verbose=2)
    ckli_status = sol.status
    timings[t, 3] = perf_counter() - ts
    nfevs[t, 0] = sol.nfev
    print(f'CKLI optimality: {sol.optimality : g}')

    uxi[t] = sol.x[:Nuxi]
    Yxi[t] = sol.x[Nuxi:]
    uest[t] = um[t] + Psiu[t] @ uxi[t]
    Yest[t] = Ym[t] + PsiY[t] @ Yxi[t]

rel_errors[:, 1] = np.array([rl2e(Yest[t], Yref) for t in range(num_trials)])
abs_errors[:, 1] = np.array([infe(Yest[t], Yref) for t in range(num_trials)])

print(f"PICKLE: {timings[:, 3]} s")
print(f"PICKLE\trelative L2 error: {rel_errors[:, 1]}")
print(f"PICKLE\tabsolute Infinity error: {abs_errors[:, 1]}")

In [ ]:
# MAP H1 estimate
Lreg = mapest.compute_Lreg(geom)
for t in range(num_trials):
    if Neumann_sd == 0:
        loss = mapest.LossVec(Nc, Nc, iuobs, uobs, iYobs[t], Yobs[t], gamma_map, Lreg) # H1 regularization
        dasa = DASAExpLM(loss.val, loss.grad_u, loss.grad_Y, prob.solve, prob.residual_sens_u, prob.residual_sens_Y)
        ts = perf_counter()
        sol = spo.least_squares(dasa.obj, np.zeros(Nc), jac=dasa.grad, method=lsq_method, verbose=2)
        Yest_MAPH1[t] = sol.x
    else:
        loss = mapest.LossVecWithFlux(Nc, Nc, Nq, iuobs, uobs, iYobs[t], Yobs[t], gamma_map, Lreg) # H1 regularization
        dasa = DASAExpLMWithFlux(Nc, loss.val, loss.grad_u, loss.grad_p, prob.solve, prob.residual_sens_u, prob.residual_sens_p)
        ts = perf_counter()
        sol = spo.least_squares(dasa.obj, np.zeros(Nc + Nq), jac=dasa.grad, method=lsq_method, verbose=2)
        Yest_MAPH1[t] = sol.x[:Nc]
        q_MAPH1[t] = sol.x[Nc:]
    MAP_status = sol.status
    timings[t, 4] = perf_counter() - ts
    nfevs[t, 1] = sol.nfev
    print(f'MAP status: {MAP_status}, message: {sol.message}')

rel_errors[:, 2] = np.array([rl2e(Yest_MAPH1[t], Yref) for t in range(num_trials)])
abs_errors[:, 2] = np.array([infe(Yest_MAPH1[t], Yref) for t in range(num_trials)])

print(f"MAP: {timings[:, 4]} s")
print(f"MAP\trelative L2 error: {rel_errors[:, 2]}")
print(f"MAP\tabsolute infinity error: {abs_errors[:, 2]}")

In [ ]:
np.savetxt(results_path + f'iYobs/iYobs_{exp}.txt', iYobs.astype(int), fmt='%i')
np.savetxt(results_path + f'timings/timings_{exp}.txt', timings)
np.savetxt(results_path + f'nfevs/nfevs_{exp}.txt', nfevs.astype(int), fmt='%i')
np.savetxt(results_path + f'rel_errors/rel_errors_{exp}.txt', rel_errors)
np.savetxt(results_path + f'abs_errors/abs_errors_{exp}.txt', abs_errors)
np.savetxt(results_path + f'YGPR/YGPR_{exp}.txt', Yref)
np.savetxt(results_path + f'YPICKLE/YPICKLE_{exp}.txt', Yest)
np.savetxt(results_path + f'YMAP/YMAP_{exp}.txt', Yest_MAPH1)